In [1]:
import os
import requests
import json
import time
import numpy as np
import pandas as pd
from pprint import pprint
import datetime

In [2]:
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y%m%d')
yesterday

'20230207'

In [3]:
today = datetime.datetime.now().strftime('%Y%m%d')
today

'20230208'

In [4]:
# time_stamp = time.time()    #取出时间戳
# time_stamp = int(time_stamp*1000)   #将10位整数变为13位整数
# print(time_stamp)
# page = 10 #input("请输入你想要爬取的页数:")


# # url = 'http://41.push2.eastmoney.com/api/qt/clist/get'
# url = 'http://71.push2.eastmoney.com/api/qt/clist/get?'
headers = {
    'Referer':
    'http://quote.eastmoney.com/',
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36'
}

# #这个参数列表
# datas = {
#     'cb': f'jQuery112403210602285843305_{time_stamp}',
#     'pn': f'{page}',
#     'pz': '20',
#     'po': '1',
#     'np': '1',
#     'ut': 'bd1d9ddb04089700cf9c27f6f7426281',
#     'fltt': '2',
#     'invt': '2',
#     'fid': 'f3',
#     'fs': 'm:1 s:2',
#     'fields':
#     'f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152',
#     '_': f'{time_stamp}'
# }

# 爬取今天数据

In [5]:
# url = """http://71.push2.eastmoney.com/api/qt/clist/get?cb=jQuery112403210602285843305_1675345303710&pn=%s&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=|0|0|0|web&fid=f3&fs=m:0+t:6,m:0+t:80,m:1+t:2,m:1+t:23,m:0+t:81+s:2048&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1675345303757"""

In [6]:
url = """http://11.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124045425929349494565_1675652972800&pn=%s&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=5556335897207406|0|0|0|web&fid=f3&fs=m:0+t:6,m:0+t:80,m:1+t:2,m:1+t:23,m:0+t:81+s:2048&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1675652972804"""

In [7]:
def get_data_pn(url, pn):
    url = url % pn
    # print(url)
    # jsondata = requests.get(url=url, headers=headers, params=datas).text  # 输出文本
    jsondata = requests.get(url=url, headers=headers).text  # 输出文本

    data_pn_li = []
    start_data = jsondata.index('{"rc":0,')  # 取出前面文本
    end_data = jsondata.index('}]}}') + len('}]}}')  # 取出后面文本
    for json_data in json.loads(jsondata[start_data:end_data])['data']['diff']:
        info_tmp = []
        info_tmp.append(json_data['f12'])  # 名称
        info_tmp.append(json_data['f14'])  # 代码
        info_tmp.append(json_data['f2'])  # 最新价
        info_tmp.append(json_data['f17'])  # 今开
        info_tmp.append(json_data['f18'])  # 昨收
        # info_tmp.append(json_data['f15'])  # 最高
        # info_tmp.append(json_data['f16'])  # 最低
        # info_tmp.append(json_data['f8'])  # 换手率
        # info_tmp.append(json_data['f23'])  # 市净率
        data_pn_li.append(info_tmp)
    return data_pn_li

In [8]:
# data_all_li

In [9]:
data_res_file = "data/all_data_%s.json" % today
if not os.path.exists(data_res_file):
    print(">>>初始化结果文件")
    pn_li = [pn for pn in range(1, 265)]
    data_pn_res = []

    all_data_json = {}
    all_data_json["pn"] = pn_li
    all_data_json['data'] = data_pn_res
    json.dump(all_data_json, open(data_res_file, "w", encoding="utf-8"))
else:
    print(">>>读取结果数据")
    all_data_json = json.load(open(data_res_file, "r", encoding="utf-8"))
    pn_li = all_data_json["pn"]
    data_pn_res = all_data_json['data']

>>>初始化结果文件


In [10]:
pn_per = []
for pn in pn_li:
    try:
        time.sleep(0.2)
        print(">>> 正在处理第%s页" % pn)
        data_pn_tmp = get_data_pn(url, pn)
        pn_per.append(pn)
        data_pn_res.extend(data_pn_tmp)
    except Exception as e:
        print(">>> 异常", pn, e)
        all_data_json["pn"] = list(set(pn_li) - set(pn_per))
        all_data_json["data"] = data_pn_res
        json.dump(all_data_json, open(data_res_file, "w", encoding="utf-8"))

print("任务完成".center(20, "*"))
all_data_json["pn"] = list(set(pn_li) - set(pn_per))
all_data_json["data"] = data_pn_res
json.dump(all_data_json, open(data_res_file, "w", encoding="utf-8"))

>>> 正在处理第1页
>>> 正在处理第2页
>>> 正在处理第3页
>>> 正在处理第4页
>>> 正在处理第5页
>>> 正在处理第6页
>>> 正在处理第7页
>>> 正在处理第8页
>>> 正在处理第9页
>>> 正在处理第10页
>>> 正在处理第11页
>>> 正在处理第12页
>>> 正在处理第13页
>>> 正在处理第14页
>>> 正在处理第15页
>>> 正在处理第16页
>>> 正在处理第17页
>>> 正在处理第18页
>>> 正在处理第19页
>>> 正在处理第20页
>>> 正在处理第21页
>>> 正在处理第22页
>>> 正在处理第23页
>>> 正在处理第24页
>>> 正在处理第25页
>>> 正在处理第26页
>>> 正在处理第27页
>>> 正在处理第28页
>>> 正在处理第29页
>>> 正在处理第30页
>>> 正在处理第31页
>>> 正在处理第32页
>>> 正在处理第33页
>>> 正在处理第34页
>>> 正在处理第35页
>>> 正在处理第36页
>>> 正在处理第37页
>>> 正在处理第38页
>>> 正在处理第39页
>>> 正在处理第40页
>>> 正在处理第41页
>>> 正在处理第42页
>>> 正在处理第43页
>>> 正在处理第44页
>>> 正在处理第45页
>>> 正在处理第46页
>>> 正在处理第47页
>>> 正在处理第48页
>>> 正在处理第49页
>>> 正在处理第50页
>>> 正在处理第51页
>>> 正在处理第52页
>>> 正在处理第53页
>>> 正在处理第54页
>>> 正在处理第55页
>>> 正在处理第56页
>>> 正在处理第57页
>>> 正在处理第58页
>>> 正在处理第59页
>>> 正在处理第60页
>>> 正在处理第61页
>>> 正在处理第62页
>>> 正在处理第63页
>>> 正在处理第64页
>>> 正在处理第65页
>>> 正在处理第66页
>>> 正在处理第67页
>>> 正在处理第68页
>>> 正在处理第69页
>>> 正在处理第70页
>>> 正在处理第71页
>>> 正在处理第72页
>>> 正在处理第73页
>>> 正在处理第74页
>>> 正在处理第75页
>>> 正在处理第76页
>>> 正在处理第77页
>>> 正在处理

In [11]:
data_today = pd.DataFrame(all_data_json["data"], columns=['代码', '名称', '最新价', '今开', '昨收'])
data_today["获取时间"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data_today.to_csv("data/all_data_" + today + ".csv", encoding="utf-8", index=False)

In [12]:
data_today

,代码,名称,最新价,今开,昨收,获取时间
0,301373,N凌玮,43.35,44,33.73,2023-02-08 23:10:50
1,301260,N格力博,38.3,37,30.85,2023-02-08 23:10:50
2,300317,珈伟新能,6.88,5.88,5.73,2023-02-08 23:10:50
3,688787,海天瑞声,189.6,158.92,158,2023-02-08 23:10:50
4,300114,中航电测,26.34,26.34,21.95,2023-02-08 23:10:50
...,...,...,...,...,...,...
5275,688027,国盾量子,142.59,153,156.16,2023-02-08 23:10:50
5276,300071,福石控股,5.93,6.25,6.5,2023-02-08 23:10:50
5277,688229,博睿数据,44.9,48.52,49.26,2023-02-08 23:10:50
5278,002512,达华智能,4.38,4.59,4.81,2023-02-08 23:10:50


# 读取昨天数据

In [13]:
data_yesterday = pd.read_csv("data/all_data_" + yesterday + ".csv", encoding="utf-8", dtype={'代码': object})
data_yesterday.columns = ['代码', '名称', '最新价', '昨开', '前收', '获取时间']
# data_yesterday

In [14]:
data_today = pd.read_csv("data/all_data_" + today + ".csv", encoding="utf-8", dtype={'代码': object})
# data_today

In [15]:
data_all_new = pd.merge(data_today, data_yesterday[['代码',  '昨开']], on=['代码'], how="inner")

In [16]:
data_today['代码'].value_counts()

600001    1
603129    1
600282    1
300110    1
002371    1
         ..
601326    1
600900    1
603052    1
002188    1
601568    1
Name: 代码, Length: 5280, dtype: int64

In [17]:
data_all_new = data_all_new[data_all_new['最新价'] != "-"]
data_all_new = data_all_new[data_all_new['今开'] != "-"]
data_all_new = data_all_new[data_all_new['昨开'] != "-"]
data_all_new = data_all_new[data_all_new['昨收'] != "-"]
data_all_new = data_all_new[['代码', '名称', '最新价', '今开', '昨开', '昨收', '获取时间']]

for col in ['最新价', '今开', '昨开', '昨收']:
    data_all_new[col] = data_all_new[col].astype(np.float64)

In [18]:
data_all_new["A"] = (data_all_new["最新价"] - data_all_new["昨收"]) / (data_all_new["昨开"] - data_all_new["昨收"])
# data_all_new["A"] = data_all_new["A"].apply(lambda x: format(x, '.2%'))

In [19]:
data_all_new

,代码,名称,最新价,今开,昨开,昨收,获取时间,A
2,300317,珈伟新能,6.88,5.88,5.42,5.73,2023-02-08 23:10:50,-3.709677
3,688787,海天瑞声,189.60,158.92,136.00,158.00,2023-02-08 23:10:50,-1.436364
4,300114,中航电测,26.34,26.34,21.95,21.95,2023-02-08 23:10:50,inf
6,300309,*ST吉艾,1.04,0.86,0.91,0.87,2023-02-08 23:10:50,4.250000
7,300473,德尔股份,23.96,21.00,18.56,20.06,2023-02-08 23:10:50,-2.600000
...,...,...,...,...,...,...,...,...
5271,688027,国盾量子,142.59,153.00,139.18,156.16,2023-02-08 23:10:50,0.799176
5272,300071,福石控股,5.93,6.25,5.91,6.50,2023-02-08 23:10:50,0.966102
5273,688229,博睿数据,44.90,48.52,49.17,49.26,2023-02-08 23:10:50,48.444444
5274,002512,达华智能,4.38,4.59,4.46,4.81,2023-02-08 23:10:50,1.228571


In [20]:
flag_ss = data_all_new['代码'].str.startswith("0") | data_all_new['代码'].str.startswith("6")
data_all_new = data_all_new[flag_ss]
data_all_new = data_all_new[data_all_new['最新价']-data_all_new['今开'] > 0]
data_all_new = data_all_new[data_all_new['A'].between(0.5, 1.5)]
data_all_new['A'] = data_all_new["A"].round(2)

In [21]:
# 读取股票基本信息
name_list = pd.read_csv("data/name_list_20230202.csv",
                        encoding="utf-8", dtype={'股票代码': object})
name_list.rename(columns={"股票代码": "代码"}, inplace=True)
name_list

,代码,名称,地区,行业,版块,上市日期
0,000001,平安银行,深圳,银行,主板,19910403
1,000002,万科A,深圳,全国地产,主板,19910129
2,000004,ST国华,深圳,软件服务,主板,19910114
3,000005,ST星源,深圳,环境保护,主板,19901210
4,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...
5074,873223,荣亿精密,NaN,NaN,北交所,20220609
5075,873305,九菱科技,NaN,NaN,北交所,20221221
5076,873339,恒太照明,NaN,NaN,北交所,20221117
5077,873527,夜光明,NaN,NaN,北交所,20221027


In [22]:
name_list['代码'].value_counts()

603667    1
301024    1
002080    1
000416    1
301111    1
         ..
601326    1
600900    1
600170    1
002153    1
601568    1
Name: 代码, Length: 5079, dtype: int64

In [23]:
data_all_new = pd.merge(data_all_new, name_list[['代码', '地区', '行业', '版块']], on=['代码'], how="left")
data_all_new = data_all_new[['代码', '名称', '地区',
                             '行业', '版块', '最新价', '今开', '昨开', '昨收', '获取时间', 'A']]

In [24]:
data_all_new = data_all_new.sort_values(['代码'])
data_all_new["代码"] = "\t" + data_all_new["代码"]

In [25]:
data_all_new

,代码,名称,地区,行业,版块,最新价,今开,昨开,昨收,获取时间,A
30,\t000007,全新好,深圳,其他商业,主板,7.71,7.63,7.69,7.62,2023-02-08 23:10:50,1.29
94,\t000096,广聚能源,深圳,石油贸易,主板,8.52,8.50,8.53,8.51,2023-02-08 23:10:50,0.50
92,\t000510,新金路,四川,化工原料,主板,6.46,6.44,6.47,6.45,2023-02-08 23:10:50,0.50
65,\t000757,浩物股份,四川,汽车配件,主板,4.48,4.47,4.48,4.46,2023-02-08 23:10:50,1.00
61,\t000786,北新建材,北京,其他建材,主板,28.01,27.88,27.97,27.86,2023-02-08 23:10:50,1.36
...,...,...,...,...,...,...,...,...,...,...,...
15,\t688625,呈和科技,广东,化工原料,科创板,54.95,54.00,55.78,54.00,2023-02-08 23:10:50,0.53
10,\t688656,浩欧博,江苏,医疗保健,科创板,38.73,37.82,38.47,37.78,2023-02-08 23:10:50,1.38
100,\t688670,金迪克,江苏,生物制药,科创板,47.78,47.49,47.76,47.89,2023-02-08 23:10:50,0.85
19,\t688789,宏华数科,浙江,专用机械,科创板,186.35,184.38,186.50,183.52,2023-02-08 23:10:50,0.95


In [26]:
data_all_new.to_excel("data/data_all_result_" + today + ".xlsx", index=False)